In [ ]:
import os

dir_train_query_path='/content/drive/MyDrive/VOTE_DOC/ds_sosedi/train'
dir_validation_gallery_path='/content/drive/MyDrive/VOTE_DOC/ds_sosedi/train'

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {'label': [],
        'src_file': [],
        'split': [],
        'is_query': [],
        'is_gallery': [],
        'path': []
        }

df_vote = pd.DataFrame(data)


In [ ]:
label2id={'za': 0,'protiv':1,'negolos':2,'isporchen':3,'vozderj':4}

In [ ]:


is_query=None
is_gallery=None

for root, label, files in os.walk(dir_train_query_path, topdown=False):
  for name in files:
    if name.endswith('jpg'):
      path = os.path.join(root, name)
      label = root.split('/')[-1]
      if label=='':
        print('break:::',root,name)
        break

      new_row = {'label': label2id[label],
             'src_file': name,
             'split': 'train',
             'is_query': is_query,
             'is_gallery': is_gallery,
             'path': path
             }

      df_vote.loc[len(df_vote)] = new_row


In [ ]:
is_query=0
is_gallery=1

for root, label, files in os.walk(dir_validation_gallery_path, topdown=False):
  for name in files:
    if name.endswith('jpg'):
      path = os.path.join(root, name)
      label = root.split('/')[-1]
      if label=='':
        print('break:::',root,name)
        break

      new_row = {'label': label2id[label],
             'src_file': name,
             'split': 'validation',
             'is_query': is_query,
             'is_gallery': is_gallery,
             'path': path
             }
      if (is_query==1):
        is_query=0
        is_gallery=1
      else:
        is_query=1
        is_gallery=0

      df_vote.loc[len(df_vote)] = new_row


In [ ]:
%cd /content/drive/MyDrive/VOTE_DOC

/content/drive/MyDrive/VOTE_DOC


In [ ]:

# saving the dataframe
df_vote.to_csv('df_vote.csv')


In [ ]:
df_vote

,label,src_file,split,is_query,is_gallery,path
0,0,za_za_100.jpg,train,None,None,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
1,0,za_za_101.jpg,train,None,None,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
2,0,za_za_102.jpg,train,None,None,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
3,0,za_za_103.jpg,train,None,None,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
4,0,za_za_104.jpg,train,None,None,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
...,...,...,...,...,...,...
3123,4,204.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
3124,4,205.jpg,validation,0,1,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
3125,4,206.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...
3126,4,207.jpg,validation,0,1,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...


In [ ]:
!pip install -U open-metric-learning

In [ ]:
columns=['label','path','split','is_query', 'is_gallery','category']
header="label,path,split,is_query,is_gallery"

In [ ]:

!! pip install pytorch-metric-learning
!! pip install pillow==9.0.0

['Requirement already satisfied: pillow==9.0.0 in /usr/local/lib/python3.10/dist-packages (9.0.0)']

In [ ]:

from tqdm import tqdm
from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.datasets.base import DatasetQueryGallery
from oml.metrics.embeddings import EmbeddingMetrics
from oml.utils.dataframe_format import check_retrieval_dataframe_format
from oml.utils.misc_torch import PCA
from oml.inference.flat import inference_on_images
from oml.registry.transforms import get_transforms_for_pretrained
from oml.utils.misc_torch import pairwise_dist
from pytorch_metric_learning import losses, distances, reducers, miners
import pandas as pd
from oml.transforms.images.torchvision import get_normalisation_resize_torch
from oml.models.meta.siamese import ConcatSiamese
from oml.miners.pairs import PairsMiner
from oml.inference.flat import inference_on_dataframe
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from oml.retrieval.postprocessors.pairwise import PairwiseImagesPostprocessor
from pprint import pprint



In [ ]:
model_types=['vits16_dino', 'vits8_dino', 'vitb16_dino', 'vitb8_dino', 'vits16_inshop', 'vits16_sop', 'vits16_cub', 'vits16_cars']
modeltype='vitb8_dino'
arch='vitb8'

In [ ]:
import torch

In [ ]:
from pathlib import Path
from oml.utils.dataframe_format import check_retrieval_dataframe_format

In [ ]:
import pathlib

vote_path='/'
vote_path=pathlib.PosixPath(vote_path)
p=vote_path



check_retrieval_dataframe_format(df=df_vote, dataset_root=p)

In [ ]:
from oml.const import CKPT_SAVE_ROOT as CKPT_DIR, MOCK_DATASET_PATH as DATA_DIR
from oml.models import ViTExtractor
from oml.registry.transforms import get_transforms_for_pretrained


# first time
#model = ViTExtractor.from_pretrained("vitb8_dino")
model = ViTExtractor.from_pretrained("vits16_dino")

# second time
#model = ViTExtractor(weights=CKPT_DIR / "vits16_dino.ckpt", arch="vits16", normalise_features=False)
#transform, im_reader = get_transforms_for_pretrained("vitb8_dino")
transform, im_reader = get_transforms_for_pretrained("vits16_dino")

https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth


In [ ]:
# тут НЕ сохраненные на зрачках веса грузятся
#model = ViTExtractor(modeltype, arch=arch, normalise_features=False).train()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
#transform = get_normalisation_resize_torch(im_size=im_size)
train_dataset = DatasetWithLabels(df_vote, dataset_root=p,transform=transform)
device = torch.device("cuda")
model.to(device)
threshold=0.4

# PML specific
distance = distances.LpDistance(p=2)
reducer = reducers.ThresholdReducer(low=0)
criterion = losses.TripletMarginLoss()

miner = miners.TripletMarginMiner(margin=threshold, distance=distance, type_of_triplets="hard")

sampler = BalanceSampler(train_dataset.get_labels(), n_labels=2, n_instances=2)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=sampler)

# Одна эпоха потом посмотрим когда метрики построим может и еще надо
n_epoch=1

for i in range(n_epoch):
  for batch in tqdm(train_loader):
    embeddings = model(batch["input_tensors"].to(device))
    loss = criterion(embeddings, batch["labels"], miner(embeddings, batch["labels"]))  # PML specific
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




100%|██████████| 2/2 [00:00<00:00, 13.95it/s]


In [ ]:
miner = miners.TripletMarginMiner(margin=threshold, distance=distance, type_of_triplets="hard")

In [ ]:
# создаем калькулятор метрик, в котором будем накапливать вектора

val_dataset = DatasetQueryGallery(df_vote, dataset_root=p,transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4)

calculator = EmbeddingMetrics(map_top_k=(3,2,1),cmc_top_k =(3,2,1),precision_top_k =(3,2,1) )
calculator.setup(num_samples=len(train_dataset))

with torch.no_grad():
    for batch in tqdm(val_loader):
        batch["embeddings"] = model(batch["input_tensors"].to(device))
        calculator.update_data(batch)  # накапливаем вектора


100%|██████████| 782/782 [00:34<00:00, 22.72it/s]


In [ ]:
#vitsb8_calculator.compute_metrics()  # вычисляем метрики: cmc@k, precision@k, map@k


Metrics:
{'OVERALL': {'cmc': {1: tensor(0.8197), 2: tensor(0.8875), 3: tensor(0.9092)},
             'map': {1: tensor(0.8197), 2: tensor(0.8536), 3: tensor(0.8551)},
             'pcf': {0.5: tensor(0.0052)},
             'precision': {1: tensor(0.8197),
                           2: tensor(0.7692),
                           3: tensor(0.7408)}}}


{'OVERALL': {'cmc': {3: tensor(0.9092), 2: tensor(0.8875), 1: tensor(0.8197)},
  'precision': {3: tensor(0.7408), 2: tensor(0.7692), 1: tensor(0.8197)},
  'map': {3: tensor(0.8551), 2: tensor(0.8536), 1: tensor(0.8197)},
  'pcf': {0.5: tensor(0.0052)}}}

In [ ]:
modelpath='/content/drive/MyDrive/VOTE_DOC/df_vit9_1.pt'
torch.save(model.state_dict(), modelpath)

In [ ]:
calculator.compute_metrics()  # вычисляем метрики: cmc@k, precision@k, map@k


Metrics:
{'OVERALL': {'cmc': {1: tensor(0.8350), 2: tensor(0.8887), 3: tensor(0.9143)},
             'map': {1: tensor(0.8350), 2: tensor(0.8619), 3: tensor(0.8666)},
             'pcf': {0.5: tensor(0.0078)},
             'precision': {1: tensor(0.8350),
                           2: tensor(0.7845),
                           3: tensor(0.7447)}}}


{'OVERALL': {'cmc': {3: tensor(0.9143), 2: tensor(0.8887), 1: tensor(0.8350)},
  'precision': {3: tensor(0.7447), 2: tensor(0.7845), 1: tensor(0.8350)},
  'map': {3: tensor(0.8666), 2: tensor(0.8619), 1: tensor(0.8350)},
  'pcf': {0.5: tensor(0.0078)}}}

In [ ]:
modelpath='/content/drive/MyDrive/VOTE_DOC/df_vits16_1.pt'
torch.save(model.state_dict(), modelpath)

In [ ]:
#metrics = calculator.compute_metrics()  # вычисляем метрики: cmc@k, precision@k, map@k

In [ ]:
#metrics = calculator.compute_metrics()  # вычисляем метрики: cmc@k, precision@k, map@k

In [ ]:
df = df_vote

In [ ]:
# Смысл такой что по запросам query мы ищем ближайший (похоже 1)
# но только среди элементов gallery
queries=[]
df_val=df[df['split']=='validation']
df_val["path"] = df_val["path"].apply(lambda x: p / x)
queries = df_val[df_val["is_query"]==1.0]["path"].tolist()
galleries = df_val[df_val["is_gallery"]==1.0]["path"].tolist()

args = {"num_workers": 0, "batch_size": 8}
# эмбеддинги запросов и галереи
features_queries = inference_on_images(model, paths=queries, transform=transform, **args)
features_galleries = inference_on_images(model, paths=galleries, transform=transform, **args)

use_knn = False
top_k = 1
if use_knn:
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(algorithm="auto", p=5)
    knn.fit(features_galleries)
    dists, ii_closest = knn.kneighbors(features_queries, n_neighbors=top_k, return_distance=True)

else:
    dist_mat = pairwise_dist(x1=features_queries, x2=features_galleries)
    dists, ii_closest = torch.topk(dist_mat, dim=1, k=top_k, largest=False)

#print(f"Top {top_k} items closest to queries are:\n {ii_closest}")


<ipython-input-60-a55db1ff3703>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val["path"] = df_val["path"].apply(lambda x: p / x)


In [ ]:
df_queries = df_val[df_val["is_query"]==1.0]
df_galleries = df_val[df_val["is_gallery"]==1.0]

In [ ]:
ppath=[]
predict=[]
dist=[]




for ic,id in zip(ii_closest.tolist(),dists.tolist()):
  predict.append(df_galleries.iloc[ic[0]]['label'])
  ppath.append((df_galleries.iloc[ic[0]]['src_file']))
  dist.append(id[0])


In [ ]:
df_queries['predict']=predict
df_queries['ppath']=ppath
df_queries['dist']=dist


<ipython-input-67-76569fcf27f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict']=predict
<ipython-input-67-76569fcf27f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['ppath']=ppath
<ipython-input-67-76569fcf27f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [ ]:
df_match=df_queries[df_queries['label']==df_queries['predict']]

In [ ]:
df_match

,label,src_file,split,is_query,is_gallery,path,predict,ppath,dist
1565,0,za_za_101.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,za_za_102.jpg,29.081320
1567,0,za_za_103.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,103.jpg,0.000000
1569,0,za_za_105.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,105.jpg,0.000000
1571,0,za_za_107.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,za_za_157.jpg,22.564795
1573,0,za_za_109.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,109.jpg,0.000000
...,...,...,...,...,...,...,...,...,...
3115,4,196.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,4,197.jpg,12.644323
3117,4,198.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,4,197.jpg,9.140558
3119,4,200.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,4,197.jpg,13.258473
3121,4,202.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,4,203.jpg,23.847988


In [ ]:
df_err=df_queries[df_queries['label']!=df_queries['predict']]

In [ ]:
df_err

,label,src_file,split,is_query,is_gallery,path,predict,ppath,dist
1605,0,za_za_138.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,2,negolos_negolos_35.jpg,21.088043
1609,0,za_za_141.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,1,protiv_protiv_45.jpg,20.853283
1611,0,za_za_143.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,1,protiv_protiv_85.jpg,24.155270
1615,0,za_za_147.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,1,protiv_protiv_85.jpg,23.060276
1643,0,za_za_172.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,4,37.jpg,28.877874
...,...,...,...,...,...,...,...,...,...
3085,4,166.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,334.jpg,16.152306
3089,4,170.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,569.jpg,16.312441
3109,4,190.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,547.jpg,18.862553
3125,4,206.jpg,validation,1,0,/content/drive/MyDrive/VOTE_DOC/ds_sosedi/trai...,0,517.jpg,14.066631


In [ ]:
# vitsb8 642 match, 140 no match

In [ ]:
# vits16 653 match 129 no match

In [ ]:
p = 1 - 129/(653+129)

In [ ]:
p # точность vits16

0.8350383631713555

In [ ]:
label2id

{'za': 0, 'protiv': 1, 'negolos': 2, 'isporchen': 3, 'vozderj': 4}